<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_local_search_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Faiss 로컬 실험 코드

In [ ]:
!pip install faiss-cpu #서버 업로드 시 삭제
!pip install -U sentence-transformers #서버 업로드 시 삭제
!pip install datasets

import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
import torch
import json
import torch.nn as nn
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from google.colab import drive
from transformers import AutoModel, AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments, BertForMaskedLM, Trainer, TrainerCallback
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyli

In [ ]:
#train_models에서 학습시킨 KoSimCSE_BERT모델의 pooler 층이 제대로 저장되지 않음.
HUGGINGFACE_MODEL_PATH = 'BM-K/KoSimCSE-bert-multitask'

model = AutoModel.from_pretrained(HUGGINGFACE_MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

model.safetensors:  55%|#####4    | 241M/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# #모델 불러오기
# model_path = '/content/drive/My Drive/Pretrained_Model_sentence_embed'

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModel.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Pretrained_Model_sentence_embed and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
import ast
#로컬용 FAISS 코드
#데이터 불러오기
test_file_path = '/content/drive/My Drive/sentence_embed_result_short.csv'
df = pd.read_csv(test_file_path)

#csv파일로 저장할 때 쉼표를 살리기 위해 json.dumps로 저장했기 때문에 json.loads 사용
df['embedding'] = df['embedding'].apply(json.loads)
encoded_data = torch.tensor(df['embedding'].tolist())
data = df.drop(['embedding'], axis = 1)

#FAISS에 등록
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
print(encoded_data)
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, 'question-answer')


tensor([[-0.4031,  1.0123,  0.0415,  ...,  0.5769,  1.1547,  1.3611],
        [-0.3555, -0.2534, -0.1154,  ..., -0.6810, -0.1860,  1.0780],
        [ 0.4558, -0.1636,  0.4740,  ..., -0.1555,  0.1887, -0.3767],
        ...,
        [ 0.4605, -0.2990,  0.1759,  ..., -0.3592, -0.4286, -0.0604],
        [-0.0631,  0.3772,  0.1570,  ..., -0.4012, -0.4965,  0.1990],
        [-0.8053,  0.7701,  0.3127,  ..., -0.1007,  0.1395,  0.6081]])


In [ ]:
#검색 함수에서 사용할 사용자 입력 인코딩 함수
def encode(query):
  inputs = tokenizer(query, padding = True, truncation = True, return_tensors = 'pt')

  with torch.no_grad():
        outputs = model(**inputs)

  #배치 단위가 아니라 단일 문장이기 때문에 mean pooling을 할 때 dim = 0으로 설정
  mean_pooling = torch.mean(outputs.last_hidden_state, dim = 0)
  return mean_pooling

In [ ]:
def search(query):
  t = time.time()
  query_vector = encode(query)
  k = 20
  top_k = index.search(query_vector, k)
  print('total time: {}'.format(time.time() - t))

  return [{'question': data.iloc[_id]['question'], 'answer': data.iloc[_id]['answer'], 'question_summary': data.iloc[_id]['question_summary'], 'answer_summary': data.iloc[_id]['answer_summary'], 'questioner' : data.iloc[_id]['questioner'], 'respondent' : data.iloc[_id]['respondent'], 'date' : data.iloc[_id]['date'], 'url' : data.iloc[_id]['url']} for _id in top_k[1].tolist()[0]]

In [ ]:
query = str(input())
results = search(query)
print('results :')

for result in results:
  print('\t Question summary :', result['question_summary'], '\n')
  print('\t Answer summary :', result['answer_summary'], '\n')
  print('\t Question :', result['question'], '\n')
  print('\t Answer :', result['answer'], '\n')
  print('\t Questioner :', result['questioner'], '\n')
  print('\t Respondent :', result['respondent'], '\n')
  print('\t Date :', result['date'], '\n')
  print('\t URL :', result['url'], '\n\n\n')

expo 툴
total time: 0.15009808540344238
results :
	 Question summary : 노코드 사이트에 가장 만들기 쉬윈 하이브리드앱 개발 툴 expo에 대해 조코딩님 영상 첨부 글 봤는데 혹시 접해보신분 계신가요? 

	 Answer summary : 프로덕트 디자이너입니다. 업무 시 Figma에서 Auto-Layout 기능으로 UI 화면을 구성하는데, 이 기능 구성과 매우 유사해서 구현하기 쉽고, iOS, Android, Web 한번에 구현되는데, 플랫폼마다 구현이 매끄럽지 않은 기능들이 있 

	 Question :  모두의 노코드 사이트에 가장 만들기 쉬윈 하이브리드앱 개발 툴 expo 에 대해 조코딩님 영상 첨부 글 봤는데요. 혹시 expo 접해보신분 계신가요? 플러터보다 쉬운 개발 툴 

	 Answer : [' 비개발자를 위한 원데이 클래스(?) 이런 것 통해 하루 써본적 있네요.해당 클래스에서 준비해 놓은 dooboo-cli 통해서 해서 그런지, 6시간만에 메인화면과 상세화면 만들 수 있었네요. ( 개발툴 설치, 환경 설정, 개념 이해 포함 )참고로 저는 프로덕트 디자이너입니다. 업무 시 Figma에서 Auto-Layout 기능으로 UI 화면을 구성하는데, 이 기능 구성과 매우 유사해서 구현하기 쉽더라구요!참고로 iOS, Android, Web 한번에 구현되는데, 플랫폼마다 구현이 매끄럽지 않은 기능들이 있더라구요 ( 그림자 효과 등 )제 답변이 도움 되셨으면 하네요 ', ' GUI가 제공되는 노코드,로우코드 툴이 아니라 개발 프레임워크인 것 같습니다', ' 네 expo는 플러터 같은 개발 프레임워크에요! 맞아요!', ' 러닝커브가 가장 낮은 개발 프레임워크군요 버블은 GUI 제공되는 로우코드'] 

	 Questioner :  호수 

	 Respondent : [' 흰둥', ' 배문형 / 프리랜스', ' 흰둥', ' 호수'] 

	 Date : 2023-08-02 

	 URL : nan 



	 Que